<a href="https://colab.research.google.com/github/lankston-consulting/rpms-cutter/blob/main/rpms_cutout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are some libraries that will need to be downloaded if the next cell throws some ModuleNotFound errors. Delete the # of the line who is missing and run the cell.

In [ ]:
%%capture
!pip install rasterio
!pip install rioxarray

# **Library Imports and Back End Setup**
Nothing needs to be changed here, this cell just imports all the libraries our code needs to run, and gets Google Drive mounted to this notebook.

You **will** need to approve this notebook to having access to your local Google Drive, it will be a little pop up when this cell gets run!

In [ ]:
import os
import datetime
import xarray as xr
import pandas as pd
import rasterio
import rioxarray
import dask
import tempfile
from google.cloud import storage
from dask.distributed import Client, LocalCluster, Lock, as_completed, fire_and_forget
# from dotenv import load_dotenv
from google.colab import drive
# !gcloud auth login

drive.mount('/content/gdrive')
cluster = LocalCluster(n_workers=8, processes=True)
client = Client(cluster)
path_template = "gs://fuelcast-data/rpms/"
gcp_path = "gs://fuelcast-data/degradation/"
# load_dotenv()

Mounted at /content/gdrive


# **Inputs.** 

start_year: The beginning year of the range you would like.

end_year: The ending year of the range.

url_to_raster: The path to where your file will be saved locally.

In [ ]:
start_year = 2021
end_year = 2022
raster_file = "bpslut4/bpslut4_wgs84.tif"

# **RPMS Extractor**
You don't need to change anything in here, just let the cell run.

In [ ]:
file_path = gcp_path + "/" + raster_file
with rasterio.Env(GOOGLE_APPLICATION_CREDENTIALS="/content/gdrive/My Drive/fuelcast-storage-credentials.json"):
  zone_ds = rasterio.open(file_path, chunks=(1024, 1024))
  bounds = zone_ds.bounds

  profile = zone_ds.profile
  profile.update(blockxsize=1024, blockysize=1024, tiled=True)

  for y in range(start_year, end_year):
      dx = rasterio.open(path_template + str(y) + "/rpms_" + str(y) + ".tif", chunks=(1, 1024, 1024), lock=False)
      op = f"rpms_{str(y)}_mean.tif"
      with rasterio.open('/content/gdrive/My Drive/rpms_'+str(y)+'.tif', 'w', **profile) as dst:
          win = dx.window(bottom=bounds.bottom, right=bounds.right, top=bounds.top, left=bounds.left)
          dat = dx.read(window=win)
          dst.write(dat)
